In [1]:
import pandas as pd
from datetime import datetime, timedelta
import os

save_dir = 'ProcessedData'

files = os.listdir('..//Data//OriginalData')
files.sort(key=lambda x: int(x[1:x.index('_')]))


In [2]:
def get_period(time_str):
    time = datetime.strptime(time_str, "%M:%S").time()
    
    for period, (start, end) in periods.items():
        if start <= time <= end:
            return period
    return None

for file in files:
    tokens = file.split('_')
    file_name = tokens[0]
    
    data_df = pd.read_csv(f'..//Data//OriginalData//{file}')
    
    data_df['DateTime'] = pd.to_datetime(data_df['DateTime'])
    data_df['Year'] = data_df['DateTime'].dt.year
    data_df['Month'] = data_df['DateTime'].dt.month
    data_df['Day'] = data_df['DateTime'].dt.day
    data_df['Hour'] = data_df['DateTime'].dt.hour
    data_df['Time'] = data_df['DateTime'].dt.strftime('%M:%S')

    period = {
        '00': ("00:00", "09:59"),
        '10': ("10:00", "19:59"),
        '20': ("20:00", "29:59"),
        '30': ("30:00", "39:59"),
        '40': ("40:00", "49:59"),
        '50': ("50:00", "59:59")
    }

    periods = {
        key: (
            datetime.strptime(value[0], "%M:%S").time(),
            datetime.strptime(value[1], "%M:%S").time()
        )
        for key, value in period.items()
    }

    data_df['Minute'] = data_df['Time'].apply(get_period)
    
    selected_features = [
    'WindSpeed(m/s)',
    'Pressure(hpa)',
    'Temperature(°C)',
    'Humidity(%)',
    'Sunlight(Lux)',
    'Power(mW)',
    ]
    
    avg_df = data_df.groupby(['Year', 'Month', 'Day', 'Hour', 'Minute'])[selected_features].mean().round(2).reset_index()
    avg_df.rename(columns=lambda x: "Avg_" + x if x in selected_features else x, inplace=True)
    
    max_df = data_df.groupby(['Year', 'Month', 'Day', 'Hour', 'Minute'])[selected_features].max().round(2).reset_index()
    max_df.rename(columns=lambda x: "Max_" + x if x in selected_features else x, inplace=True)
    
    min_df = data_df.groupby(['Year', 'Month', 'Day', 'Hour', 'Minute'])[selected_features].min().round(2).reset_index()
    min_df.rename(columns=lambda x: "Min_" + x if x in selected_features else x, inplace=True)
    
    processed_df = avg_df
    processed_df = pd.merge(processed_df, min_df , how='inner')
    processed_df = pd.merge(processed_df, max_df, how='inner' )
    
    device_id_df = data_df['LocationCode'].to_frame()
    device_id_df.rename(columns={'LocationCode' : 'Device_ID'}, inplace=True)
    
    device_id_df = device_id_df.iloc[:len(processed_df)]
    processed_df = pd.concat([device_id_df, processed_df], axis=1)
    
    processed_df.to_csv(f"..//Data//{save_dir}//{file_name}_Processed_Train.csv",index=False)
